In [1]:
import warnings

import pandas as pd
import pm4py as pm

In [2]:
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.width', 1440)
pd.set_option('display.max_columns', None)

In [4]:
log = pm.read_xes("BPI_Challenge_2019.xes")
print(f"Total cases loaded: {len(log['case:concept:name'].unique())}")

Total cases loaded: 251734


In [ ]:
activity_counts = pm.get_event_attribute_values(log, "concept:name")
print(list(activity_counts.keys()))

['Record Goods Receipt', 'Create Purchase Order Item', 'Record Invoice Receipt', 'Vendor creates invoice', 'Clear Invoice', 'Record Service Entry Sheet', 'Remove Payment Block', 'Create Purchase Requisition Item', 'Receive Order Confirmation', 'Change Quantity', 'Change Price', 'Delete Purchase Order Item', 'Change Approval for Purchase Order', 'Cancel Invoice Receipt', 'Vendor creates debit memo', 'Change Delivery Indicator', 'Cancel Goods Receipt', 'SRM: In Transfer to Execution Syst.', 'SRM: Created', 'SRM: Awaiting Approval', 'SRM: Document Completed', 'SRM: Ordered', 'SRM: Complete', 'Release Purchase Order', 'SRM: Change was Transmitted', 'Reactivate Purchase Order Item', 'Block Purchase Order Item', 'Cancel Subsequent Invoice', 'Release Purchase Requisition', 'Change Storage Location', 'Update Order Confirmation', 'SRM: Deleted', 'Record Subsequent Invoice', 'Set Payment Block', 'SRM: Transfer Failed (E.Sys.)', 'Change Currency', 'Change Final Invoice Indicator', 'SRM: Transacti

In [6]:
EXPLICIT_REWORK_ACTIVITIES = [
    'Change Quantity', 
    'Change Price', 
    'Delete Purchase Order Item', 
    'Change Approval for Purchase Order', 
    'Cancel Invoice Receipt', 
    'Cancel Goods Receipt', 
    'Reactivate Purchase Order Item', 
    'Cancel Subsequent Invoice', 
    'Change Storage Location', 
    'Change Currency', 
    'Change payment term', 
    'Change Rejection Indicator'
]
END_ACTIVITIES = ['Clear Invoice']
CASE_ID = 'case:concept:name'
ACTIVITY = 'concept:name'
TIMESTAMP = 'time:timestamp'
VENDOR = 'case:Vendor'
PURCH_GROUP = 'case:Spend area text'
CASE_DURATION = 'case_duration_days'
ALLOWABLE_LOOPS = [
    'Record Goods Receipt',
    'Record Invoice Receipt',
    'Record Service Entry Sheet',
    'Record Subsequent Invoice',
    'Create Purchase Order Item',
    'Create Purchase Requisition Item'
]

In [7]:
# grouping for proper comparison
log['context_key'] = (
    log['case:Company'] + " | " + 
    log['case:Item Category'] + " | " + 
    log['case:Spend area text'] + " | " + 
    log['case:Document Type']
)

In [8]:
log = pm.filter_end_activities(log, END_ACTIVITIES)

In [9]:
# pm.write_xes(log, 'BPI_2019_Cleared.xes')

In [10]:
print(f"Total cases left: {len(log[CASE_ID].unique())}")

Total cases left: 181328


In [ ]:
# bpmn_model = pm.discover_bpmn_inductive(log, noise_threshold=0.2)
# pm.write_bpmn(bpmn_model, 'bpmn_model.bpmn')

In [13]:
log.head(1)

,case:GR-Based Inv. Verif.,org:resource,case:Document Type,case:Goods Receipt,case:concept:name,case:Purchasing Document,case:Spend classification text,case:Item,case:Company,case:Item Category,case:Purch. Doc. Category name,concept:name,case:Vendor,Cumulative net worth (EUR),case:Item Type,case:Sub spend area text,User,case:Source,time:timestamp,case:Name,case:Spend area text,context_key
0,False,batch_00,EC Purchase order,True,2000000000_00001,2000000000,NPR,00001,companyID_0000,"3-way match, invoice before GR",Purchase order,SRM: Created,vendorID_0000,298.0,Standard,Facility Management,batch_00,sourceSystemID_0000,2018-01-02 12:53:00+00:00,vendor_0000,CAPEX & SOCS,"companyID_0000 | 3-way match, invoice before G..."


In [14]:
log[TIMESTAMP] = pd.to_datetime(log[TIMESTAMP], utc=True)
log = log.sort_values(by=[CASE_ID, TIMESTAMP]).reset_index(drop=True)

In [ ]:
# case & edge (step -> step) durations
log['next_timestamp'] = log.groupby(CASE_ID)[TIMESTAMP].shift(-1)   
log['edge_duration_days'] = (log['next_timestamp'] - log[TIMESTAMP]).dt.total_seconds() / (24 * 60 * 60)  
log['edge_duration_days'] = log['edge_duration_days'].fillna(0)
case_stats = log.groupby(CASE_ID)[TIMESTAMP].agg(['min', 'max'])
case_stats[CASE_DURATION] = (case_stats['max'] - case_stats['min']).dt.total_seconds() / (24 * 60 * 60)
log = log.merge(case_stats[[CASE_DURATION]], on=CASE_ID, how='left')
log.drop(columns=['next_timestamp'], inplace=True)

In [16]:
log.head(1)

,case:GR-Based Inv. Verif.,org:resource,case:Document Type,case:Goods Receipt,case:concept:name,case:Purchasing Document,case:Spend classification text,case:Item,case:Company,case:Item Category,case:Purch. Doc. Category name,concept:name,case:Vendor,Cumulative net worth (EUR),case:Item Type,case:Sub spend area text,User,case:Source,time:timestamp,case:Name,case:Spend area text,context_key,edge_duration_days,case_duration_days
0,False,batch_00,EC Purchase order,True,2000000000_00001,2000000000,NPR,00001,companyID_0000,"3-way match, invoice before GR",Purchase order,SRM: Created,vendorID_0000,298.0,Standard,Facility Management,batch_00,sourceSystemID_0000,2018-01-02 12:53:00+00:00,vendor_0000,CAPEX & SOCS,"companyID_0000 | 3-way match, invoice before G...",0.041667,86.009028


In [17]:
#rework cases
explicit_rework_cases = set(log[log[ACTIVITY].isin(EXPLICIT_REWORK_ACTIVITIES)][CASE_ID].unique())

log['next_activity'] = log.groupby(CASE_ID)[ACTIVITY].shift(-1)
log['is_loop'] = log[ACTIVITY] == log['next_activity']
unhelpful_loops_mask = (log['is_loop']) & (~log[ACTIVITY].isin(ALLOWABLE_LOOPS))
cases_with_loops = log[unhelpful_loops_mask][CASE_ID].unique()

rework_cases = set(explicit_rework_cases).union(set(cases_with_loops))

case_level_df = log[[CASE_ID, VENDOR, PURCH_GROUP, CASE_DURATION]].drop_duplicates(subset=CASE_ID)
case_level_df['has_rework'] = case_level_df[CASE_ID].isin(rework_cases)

log.drop(columns=['next_activity', 'is_loop'], inplace=True)

In [18]:
total_process_duration_days = case_level_df['case_duration_days'].sum()
print(f"Total duration of all cases: {total_process_duration_days:,.2f} days")

Total duration of all cases: 15,324,125.22 days


In [ ]:
#benchmark process time
df_clean = case_level_df[~case_level_df['has_rework']].copy()

benchmark_table = df_clean.groupby([VENDOR, PURCH_GROUP])[CASE_DURATION].median().reset_index()
benchmark_table.rename(columns={CASE_DURATION: 'benchmark_duration_days'}, inplace=True)

In [20]:
benchmark_table.head(1)

,case:Vendor,case:Spend area text,benchmark_duration_days
0,vendorID_0000,CAPEX & SOCS,80.235417


In [ ]:
#cost of rework
df_rework = case_level_df[case_level_df['has_rework']].copy()
df_rework = df_rework.merge(benchmark_table, on=[VENDOR, PURCH_GROUP], how='left')

global_median = df_clean[CASE_DURATION].median()
df_rework['benchmark_duration_days'] = df_rework['benchmark_duration_days'].fillna(global_median)

df_rework['rework_tax_days'] = df_rework[CASE_DURATION] - df_rework['benchmark_duration_days']
df_rework['rework_tax_days'] = df_rework['rework_tax_days'].apply(lambda x: max(x, 0))

In [22]:
rework_speedup_opportunity_days = df_rework['rework_tax_days'].sum()
rework_improvement_potential = (rework_speedup_opportunity_days / total_process_duration_days) * 100

print(f"Clean Cases: {len(df_clean)} ({len(df_clean)/len(case_level_df):.1%})")
print(f"Rework Cases: {len(df_rework)} ({len(df_rework)/len(case_level_df):.1%})")
print(f"Rework Speedup Opportunity: {rework_speedup_opportunity_days:,.2f} days")
print(f"Potential Cycle Time Reduction: {rework_improvement_potential:.2f}% of Total Time")

Clean Cases: 154722 (85.3%)
Rework Cases: 26606 (14.7%)
Rework Speedup Opportunity: 792,441.15 days
Potential Cycle Time Reduction: 5.17% of Total Time


In [23]:
print("Top 5 Rework Drivers (Vendor + Group)")
offender_stats = df_rework.groupby([VENDOR, PURCH_GROUP])['rework_tax_days'].sum().reset_index()
offender_stats['share_of_opportunity'] = (offender_stats['rework_tax_days'] / rework_speedup_opportunity_days) * 100
offender_stats = offender_stats.sort_values(by='rework_tax_days', ascending=False)
top_5 = offender_stats.head(5).copy()
top_5['rework_tax_days'] = top_5['rework_tax_days'].map('{:,.2f}'.format)
top_5['share_of_opportunity'] = top_5['share_of_opportunity'].map('{:.2f}%'.format)
print(top_5)

Top 5 Rework Drivers (Vendor + Group)
        case:Vendor    case:Spend area text rework_tax_days share_of_opportunity
721   vendorID_0670                   Sales      128,241.24               16.18%
242   vendorID_0236                   Sales       57,219.72                7.22%
1210  vendorID_1317                   Sales       31,323.71                3.95%
199   vendorID_0197  Trading & End Products       23,088.48                2.91%
77    vendorID_0118  Trading & End Products       17,304.90                2.18%


In [24]:
df_clean.head(1)

,case:concept:name,case:Vendor,case:Spend area text,case_duration_days,has_rework
0,2000000000_00001,vendorID_0000,CAPEX & SOCS,86.009028,False


In [25]:
log.head(1)

,case:GR-Based Inv. Verif.,org:resource,case:Document Type,case:Goods Receipt,case:concept:name,case:Purchasing Document,case:Spend classification text,case:Item,case:Company,case:Item Category,case:Purch. Doc. Category name,concept:name,case:Vendor,Cumulative net worth (EUR),case:Item Type,case:Sub spend area text,User,case:Source,time:timestamp,case:Name,case:Spend area text,context_key,edge_duration_days,case_duration_days
0,False,batch_00,EC Purchase order,True,2000000000_00001,2000000000,NPR,00001,companyID_0000,"3-way match, invoice before GR",Purchase order,SRM: Created,vendorID_0000,298.0,Standard,Facility Management,batch_00,sourceSystemID_0000,2018-01-02 12:53:00+00:00,vendor_0000,CAPEX & SOCS,"companyID_0000 | 3-way match, invoice before G...",0.041667,86.009028


In [26]:
clean_case_ids = df_clean[CASE_ID].unique()
clean_log = log[log[CASE_ID].isin(clean_case_ids)].copy()

In [27]:
clean_log['next_activity'] = clean_log.groupby(CASE_ID)[ACTIVITY].shift(-1)
clean_log['transition'] = clean_log[ACTIVITY] + " -> " + clean_log['next_activity']
clean_log = clean_log.dropna(subset=['transition'])

In [28]:
clean_log.head(1)

,case:GR-Based Inv. Verif.,org:resource,case:Document Type,case:Goods Receipt,case:concept:name,case:Purchasing Document,case:Spend classification text,case:Item,case:Company,case:Item Category,case:Purch. Doc. Category name,concept:name,case:Vendor,Cumulative net worth (EUR),case:Item Type,case:Sub spend area text,User,case:Source,time:timestamp,case:Name,case:Spend area text,context_key,edge_duration_days,case_duration_days,next_activity,transition
0,False,batch_00,EC Purchase order,True,2000000000_00001,2000000000,NPR,00001,companyID_0000,"3-way match, invoice before GR",Purchase order,SRM: Created,vendorID_0000,298.0,Standard,Facility Management,batch_00,sourceSystemID_0000,2018-01-02 12:53:00+00:00,vendor_0000,CAPEX & SOCS,"companyID_0000 | 3-way match, invoice before G...",0.041667,86.009028,SRM: Complete,SRM: Created -> SRM: Complete


In [29]:
edge_stats = clean_log.groupby(['transition', 'context_key'])['edge_duration_days'].agg(
    Median=lambda x: x.median(),
    P75=lambda x: x.quantile(0.75),
    count='count'
).reset_index()

In [30]:
edge_stats.head(10)

,transition,context_key,Median,P75,count
0,Block Purchase Order Item -> Clear Invoice,"companyID_0000 | 3-way match, invoice before G...",51.036111,51.565972,2
1,Change Delivery Indicator -> Change Final Invo...,"companyID_0000 | 3-way match, invoice after GR...",0.000000,0.000000,1
2,Change Delivery Indicator -> Clear Invoice,"companyID_0000 | 3-way match, invoice after GR...",41.063889,48.580903,4
3,Change Delivery Indicator -> Clear Invoice,"companyID_0000 | 3-way match, invoice after GR...",31.574306,33.874306,2
4,Change Delivery Indicator -> Clear Invoice,"companyID_0000 | 3-way match, invoice before G...",25.331250,41.884549,4
5,Change Delivery Indicator -> Clear Invoice,"companyID_0000 | 3-way match, invoice before G...",60.296528,69.863368,8
6,Change Delivery Indicator -> Clear Invoice,"companyID_0000 | 3-way match, invoice before G...",30.619792,44.878299,2
7,Change Delivery Indicator -> Clear Invoice,"companyID_0000 | 3-way match, invoice before G...",60.189583,64.507118,10
8,Change Delivery Indicator -> Clear Invoice,"companyID_0000 | 3-way match, invoice before G...",2.068056,2.068056,1
9,Change Delivery Indicator -> Clear Invoice,"companyID_0000 | 3-way match, invoice before G...",43.481250,75.432986,42


In [31]:
edge_stats['max_acceptable_days'] = edge_stats['P75']

In [32]:
clean_log = clean_log.merge(edge_stats[['transition', 'context_key', 'max_acceptable_days', 'count']], on=['transition', 'context_key'], how='left')
clean_log['variance_tax_days'] = clean_log['edge_duration_days'] - clean_log['max_acceptable_days']
clean_log['variance_tax_days'] = clean_log['variance_tax_days'].apply(lambda x: max(x, 0))

In [33]:
clean_log.head(1)

,case:GR-Based Inv. Verif.,org:resource,case:Document Type,case:Goods Receipt,case:concept:name,case:Purchasing Document,case:Spend classification text,case:Item,case:Company,case:Item Category,case:Purch. Doc. Category name,concept:name,case:Vendor,Cumulative net worth (EUR),case:Item Type,case:Sub spend area text,User,case:Source,time:timestamp,case:Name,case:Spend area text,context_key,edge_duration_days,case_duration_days,next_activity,transition,max_acceptable_days,count,variance_tax_days
0,False,batch_00,EC Purchase order,True,2000000000_00001,2000000000,NPR,00001,companyID_0000,"3-way match, invoice before GR",Purchase order,SRM: Created,vendorID_0000,298.0,Standard,Facility Management,batch_00,sourceSystemID_0000,2018-01-02 12:53:00+00:00,vendor_0000,CAPEX & SOCS,"companyID_0000 | 3-way match, invoice before G...",0.041667,86.009028,SRM: Complete,SRM: Created -> SRM: Complete,0.083333,309,0.0


In [34]:
variance_speedup_opportunity_days = clean_log['variance_tax_days'].sum()
variance_improvement_potential = (variance_speedup_opportunity_days / total_process_duration_days) * 100

print(f"Variance Speedup Opportunity: {variance_speedup_opportunity_days:,.2f} days")
print(f"Potential Cycle Time Reduction: {variance_improvement_potential:.2f}% of Total Time")

Variance Speedup Opportunity: 2,390,036.59 days
Potential Cycle Time Reduction: 15.60% of Total Time


In [35]:
slow_transitions = clean_log.groupby(['transition'])[['variance_tax_days', 'count', 'edge_duration_days']].agg(
    variance_tax_days=('variance_tax_days', 'sum'),
    avg_frequency=('count', 'mean'),
    median_duration_days=('edge_duration_days', 'median')
).reset_index()

slow_transitions['share_of_total_variance'] = (slow_transitions['variance_tax_days'] / variance_speedup_opportunity_days) * 100
slow_transitions = slow_transitions.sort_values(by='variance_tax_days', ascending=False).head(5)

print("\nTop 5 Slowest Transitions (Bottlenecks):")
print(slow_transitions.to_string(formatters={
    'variance_tax_days': '{:,.2f}'.format,
    'avg_frequency': '{:.0f}'.format,
    'median_duration_days': '{:.2f}'.format,
    'share_of_total_variance': '{:.2f}%'.format
}))


Top 5 Slowest Transitions (Bottlenecks):
                                               transition variance_tax_days avg_frequency median_duration_days share_of_total_variance
38         Record Goods Receipt -> Record Invoice Receipt        468,421.26         21583                11.98                  19.60%
45                Record Invoice Receipt -> Clear Invoice        380,914.78         30149                37.04                  15.94%
120      Vendor creates invoice -> Record Invoice Receipt        335,642.81         27150                 4.48                  14.04%
26   Create Purchase Order Item -> Vendor creates invoice        303,231.67         23002                 5.61                  12.69%
116  Vendor creates invoice -> Create Purchase Order Item        193,118.29           889                14.37                   8.08%


In [36]:
variance_by_case = clean_log.groupby(CASE_ID)['variance_tax_days'].sum().reset_index()
df_clean = df_clean.merge(variance_by_case, on=CASE_ID, how='left')
df_clean['variance_tax_days'] = df_clean['variance_tax_days'].fillna(0)

In [37]:
clean_log.head(1)

,case:GR-Based Inv. Verif.,org:resource,case:Document Type,case:Goods Receipt,case:concept:name,case:Purchasing Document,case:Spend classification text,case:Item,case:Company,case:Item Category,case:Purch. Doc. Category name,concept:name,case:Vendor,Cumulative net worth (EUR),case:Item Type,case:Sub spend area text,User,case:Source,time:timestamp,case:Name,case:Spend area text,context_key,edge_duration_days,case_duration_days,next_activity,transition,max_acceptable_days,count,variance_tax_days
0,False,batch_00,EC Purchase order,True,2000000000_00001,2000000000,NPR,00001,companyID_0000,"3-way match, invoice before GR",Purchase order,SRM: Created,vendorID_0000,298.0,Standard,Facility Management,batch_00,sourceSystemID_0000,2018-01-02 12:53:00+00:00,vendor_0000,CAPEX & SOCS,"companyID_0000 | 3-way match, invoice before G...",0.041667,86.009028,SRM: Complete,SRM: Created -> SRM: Complete,0.083333,309,0.0


In [38]:
df_clean.head(1)

,case:concept:name,case:Vendor,case:Spend area text,case_duration_days,has_rework,variance_tax_days
0,2000000000_00001,vendorID_0000,CAPEX & SOCS,86.009028,False,53.988194


In [39]:
df_rework_subset = df_rework[[CASE_ID, VENDOR, PURCH_GROUP, 'rework_tax_days']].copy()
df_rework_subset['variance_tax_days'] = 0
df_rework_subset['segment'] = 'Rework'

df_clean_subset = df_clean[[CASE_ID, VENDOR, PURCH_GROUP, 'variance_tax_days']].copy()
df_clean_subset['rework_tax_days'] = 0
df_clean_subset['segment'] = 'Clean'

df_all = pd.concat([df_rework_subset, df_clean_subset], ignore_index=True)

In [40]:
blame_matrix = df_all.groupby([VENDOR, PURCH_GROUP]).agg(
    case_count=(CASE_ID, 'count'),
    total_rework_days=('rework_tax_days', 'sum'),
    total_variance_days=('variance_tax_days', 'sum')
).reset_index()

In [41]:
blame_matrix['total_recoverable_days'] = blame_matrix['total_rework_days'] + blame_matrix['total_variance_days']
blame_matrix['days_per_case'] = blame_matrix['total_recoverable_days'] / blame_matrix['case_count']

In [42]:
ZOMBIE_THRESHOLD = 365

blame_matrix['issue_type'] = blame_matrix['days_per_case'].apply(
    lambda x: 'Legacy/Zombie' if x > ZOMBIE_THRESHOLD else 'Operational Friction'
)

In [43]:
blame_matrix = blame_matrix.sort_values(by='total_recoverable_days', ascending=False)

In [44]:
print("Top 10 Problematic Combinations (Vendor + Group):")
print(blame_matrix.head(10).to_string(formatters={
    'total_rework_days': '{:,.1f}'.format,
    'total_variance_days': '{:,.1f}'.format,
    'total_recoverable_days': '{:,.1f}'.format,
    'days_per_case': '{:.2f}'.format
}))

Top 10 Problematic Combinations (Vendor + Group):
       case:Vendor    case:Spend area text  case_count total_rework_days total_variance_days total_recoverable_days days_per_case            issue_type
152  vendorID_0136               Packaging       10494          11,035.4           193,163.7              204,199.1         19.46  Operational Friction
324  vendorID_0236                   Sales        3165          57,219.7           140,159.4              197,379.1         62.36  Operational Friction
919  vendorID_0670                   Sales         105         128,241.2                61.3              128,302.5       1221.93         Legacy/Zombie
127  vendorID_0120               Packaging       10416           6,480.7           103,289.5              109,770.2         10.54  Operational Friction
138  vendorID_0127                   Sales        3569           7,566.3            93,483.0              101,049.2         28.31  Operational Friction
594  vendorID_0427  Trading & End Prod

In [45]:
zombies = blame_matrix[blame_matrix['issue_type'] == 'Legacy/Zombie']
print(f"\n--- ZOMBIE CLUSTERS DETECTED (Avg Delay > {ZOMBIE_THRESHOLD} days) ---")
print(f"Count: {len(zombies)}")
if len(zombies) > 0:
    print(zombies[[VENDOR, PURCH_GROUP, 'case_count', 'days_per_case']].head(5))


--- ZOMBIE CLUSTERS DETECTED (Avg Delay > 365 days) ---
Count: 10
        case:Vendor case:Spend area text  case_count  days_per_case
919   vendorID_0670                Sales         105    1221.928624
485   vendorID_0336            Additives          10     687.617760
1724  vendorID_1442            Logistics           2    3206.810417
355   vendorID_0253                                9     704.389468
79    vendorID_0089            Marketing           2    3133.375260


In [46]:
operational_matrix = blame_matrix[blame_matrix['issue_type'] == 'Operational Friction'].copy()

In [47]:
vendor_performance = blame_matrix.groupby(VENDOR)[['total_recoverable_days', 'case_count']].sum()
vendor_performance['avg_waste'] = vendor_performance['total_recoverable_days'] / vendor_performance['case_count']
worst_vendors = vendor_performance.sort_values(by='total_recoverable_days', ascending=False).head(5)

In [48]:
internal_performance = blame_matrix.groupby(PURCH_GROUP)[['total_recoverable_days', 'case_count']].sum()
internal_performance['avg_waste'] = internal_performance['total_recoverable_days'] / internal_performance['case_count']
worst_internal = internal_performance.sort_values(by='total_recoverable_days', ascending=False).head(5)

In [49]:
print(f"\nTop 5 Vendors by Total Waste (External Levers):")
print(worst_vendors)
print(f"\nTop 5 Purchasing Groups by Total Waste (Internal Levers):")
print(worst_internal)


Top 5 Vendors by Total Waste (External Levers):
               total_recoverable_days  case_count    avg_waste
case:Vendor                                                   
vendorID_0136           205502.518403       10506    19.560491
vendorID_0236           197379.078125        3165    62.363058
vendorID_0670           128302.505556         105  1221.928624
vendorID_0120           110703.266319       10430    10.613928
vendorID_0127           101049.245833        3569    28.313042

Top 5 Purchasing Groups by Total Waste (Internal Levers):
                        total_recoverable_days  case_count  avg_waste
case:Spend area text                                                 
Packaging                         1.151934e+06       75975  15.162012
Sales                             1.101866e+06       55324  19.916601
Trading & End Products            4.104599e+05       16810  24.417603
Additives                         1.729671e+05       12179  14.202078
CAPEX & SOCS                   

In [50]:
total_waste = blame_matrix['total_recoverable_days'].sum()
zombie_waste = zombies['total_recoverable_days'].sum()
operational_waste = operational_matrix['total_recoverable_days'].sum()
pct_reduction = (operational_waste / total_process_duration_days) * 100

In [51]:
print(f"Total Recoverable Opportunity:      {total_waste:,.0f} days")
print(f"Legacy/Data Cleanup (Zombies):  {zombie_waste:,.0f} days ({(zombie_waste/total_waste)*100:.1f}%)")
print(f"Operational Process Improvements:    {operational_waste:,.0f} days ({(operational_waste/total_waste)*100:.1f}%)")
print("-" * 50)
print(f"OPERATIONAL CYCLE TIME REDUCTION:   {pct_reduction:.1f}%")

Total Recoverable Opportunity:      3,182,478 days
Legacy/Data Cleanup (Zombies):  157,408 days (4.9%)
Operational Process Improvements:    3,025,069 days (95.1%)
--------------------------------------------------
OPERATIONAL CYCLE TIME REDUCTION:   19.7%
